# Subjective evaluation of active noise cancellation in headphones

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pnd
from scipy.stats import chi2_contingency
import functions as fn

## Dummy Data

In [ ]:
# Definiciones generales

samples_quantity = 30
measurements_per_sample = 4

# X1 = Estado de ANC (apagado o encendido)

anc_status = fn.anc_status(samples_quantity, measurements_per_sample)

# X2 = Ruido de fondo medido (65 dB o 85 dB, aproximadamente)

background_noise_mean = 64
background_noise_deviation = 2
background_noise_max = background_noise_mean + 3 * background_noise_deviation
delta_spl = 15
generated_noise_spl = background_noise_max + delta_spl

measured_noise = fn.measured_noise(background_noise_mean, background_noise_deviation, delta_spl, generated_noise_spl, samples_quantity, measurements_per_sample)

# Y1 = Nivel de ruido percibido

perceived_noise = fn.perceived_noise(samples_quantity, measurements_per_sample, anc_status, measured_noise, background_noise_max)

# Y2 = Calidad de sonido percibida

perceived_quality = fn.perceived_quality(samples_quantity, measurements_per_sample, anc_status)

## Validation

In [ ]:
# Generar lista de outliers

outliers = fn.list_outliers(anc_status, measured_noise, perceived_noise, background_noise_max, samples_quantity, measurements_per_sample)

for i in range(len(outliers)):
    print(f"Ruido de fondo percibido por sujeto {outliers[i] + 1} en cada caso:")
    print(f"[C1 C2 C3 C4] = {perceived_noise[outliers[i]]}\n")

# Sacar outliers de Y1

perceived_noise_valid = fn.remove_outliers(outliers, perceived_noise)
print(f"Cantidad de muestras válidas de Y1: {len(perceived_noise_valid)} de {len(perceived_noise)}")

# Sacar outliers de Y2

perceived_quality_valid = fn.remove_outliers(outliers, perceived_quality)
print(f"Cantidad de muestras válidas de Y2: {len(perceived_quality_valid)} de {len(perceived_quality)}")

# Sacar outliers en variables independientes para igualar tamaño de vectores

anc_status_valid = fn.remove_outliers(outliers, anc_status)
measured_noise_valid = fn.remove_outliers(outliers, measured_noise)

## Association

### Perceived Noise Level vs. ANC Status

In [ ]:
# Ravel data

anc_status_meaning, perceived_noise_meaning = fn.ravel_noise(anc_status_valid, perceived_noise_valid, measured_noise_valid, background_noise_max)

data = {
    'Estado ANC': anc_status_meaning,
    'Ruido percibido': perceived_noise_meaning
}

# Cross table

df = pnd.DataFrame(data)
cross_tab = pnd.crosstab(df['Estado ANC'], df['Ruido percibido'])

# Print table

cross_tab_aux_1 = pnd.crosstab(df['Estado ANC'], df['Ruido percibido'], margins=True, margins_name="Total")
cross_tab_aux_2  = pnd.crosstab(df['Estado ANC'], df['Ruido percibido'], normalize=True, margins=True, margins_name="Total")*100
combined_crosstab = cross_tab_aux_1.astype(str) + " (" + cross_tab_aux_2.round(2).astype(str) + "%)"
print(combined_crosstab)

# Bar graph

xtabgraph = cross_tab.plot(kind='bar')
plt.title('Nivel de ruido percibido')
plt.xlabel('Estado de ANC')
xtabgraph.set_xticklabels(cross_tab.index, rotation=0)
plt.ylabel('Frecuencia')
plt.show()

# Chi-square test

chi2, p, _, _ = chi2_contingency(cross_tab)
print("\nChi-Square Test:\n")
print(f"Chi2 value: {chi2:.3f}")
print(f"P-value: {p:.3f}")

### Perceived Sound Quality vs. ANC Status

In [ ]:
# Ravel data

anc_status_meaning, perceived_noise_meaning = fn.ravel_quality(anc_status_valid, perceived_quality_valid, measured_noise_valid, background_noise_max)

data = {
    'Estado ANC': anc_status_meaning,
    'Calidad percibida': perceived_noise_meaning
}

# Cross table

df = pnd.DataFrame(data)
cross_tab = pnd.crosstab(df['Estado ANC'], df['Calidad percibida'])

# Print table

cross_tab_aux_1 = pnd.crosstab(df['Estado ANC'], df['Calidad percibida'], margins=True, margins_name="Total")
cross_tab_aux_2  = pnd.crosstab(df['Estado ANC'], df['Calidad percibida'], normalize=True, margins=True, margins_name="Total")*100
combined_crosstab = cross_tab_aux_1.astype(str) + " (" + cross_tab_aux_2.round(2).astype(str) + "%)"
print(combined_crosstab)

# Bar graph

xtabgraph = cross_tab.plot(kind='bar')
plt.title('Calidad de sonido percibida')
plt.xlabel('Estado de ANC')
xtabgraph.set_xticklabels(cross_tab.index, rotation=0)
plt.ylabel('Frecuencia')
plt.show()

# Chi-square test

chi2, p, _, _ = chi2_contingency(cross_tab)
print("\nChi-Square Test:")
print(f"Chi2 value: {chi2:.3f}")
print(f"P-value: {p:.3f}")